In [1]:
"""
!pip install --upgrade pip
!pip install simplekml
!pip install pycollada
!pip install pandas
!pip install Pyarrow
!pip install numpy
!pip install -U ipykernel
"""
import sys
import os
import math

# os.chdir('/home/rainerth/notebooks/wind-turbine-kml/')
# os.chdir(os.path.dirname(__file__))

print(sys.path)

import collada_wt
from kml2track import kml_extract_launch_phase
from kml2track import igc_extract_launch_phase

import simplekml
# https://simplekml.readthedocs.io/en/latest/index.html

DIRECTORY_NESTING = True # Some Apps support only one level of nesting (no subfolders)

kml = simplekml.Kml()
kml.document = simplekml.Folder(name = "Windpark Bösingen")

safety_distance_meter = 50		# Minimum Distance in meter
safety_distance_factor = 1.0	# Minimum distance in rotor diameter multiplied by this factor
safety_distance_top = 100		# Minimum distance in meter to top of turbine


['/home/rainerth/prj/dfvb/windpark-boesingen', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/rainerth/venv-py311/lib/python3.11/site-packages']


In [2]:
def create_kml_cylinder (kmlfolder, cyl_center, cyl_diameter, cyl_height, cyl_color = simplekml.Color.green, cyl_name = 'cylinder', cyl_visibility = 1):

	# Mittelpunkt und Radius des Kreises
	center = cyl_center
	radius_m = cyl_diameter/2	# in m radius
	height = cyl_height

	# Umrechnung des Radius in Grad unter Berücksichtigung der Breitengrade
	radius_deg_lat = radius_m / 111139 	# Umwandlung von Metern in Grad
	radius_deg_lon = radius_deg_lat / math.cos(math.radians(center[1]))

	# Koordinaten des Kreises
	coords = []
	for i in range(360):
		theta = math.radians(i)
		x = center[0] + radius_deg_lon * math.cos(theta)
		y = center[1] + radius_deg_lat * math.sin(theta)
		coords.append((x, y, height))

	# Polygon
	pol = kmlfolder.newpolygon(
		name=cyl_name,
		outerboundaryis=coords,
		altitudemode=simplekml.AltitudeMode.relativetoground,
		extrude=1,
		visibility=cyl_visibility
	)

	# Farbe und Transparenz des Polygons
	pol.style.linestyle.width = 1
	pol.style.linestyle.color = cyl_color
	#pol.style.linestyle.color = simplekml.Color.changealphaint(100, cyl_color)

	#pol.style.polystyle.color = cyl_color
	pol.style.polystyle.color = simplekml.Color.changealphaint(200, cyl_color)
	#pol.style.polystyle.colormode = simplekml.ColorMode.random
	pol.style.polystyle.outline = 1
	pol.style.polystyle.fill = 1

	return


In [3]:
# create turbine
# https://www.wind-turbine-models.com/turbines/2425-vestas-v172-7.2-enventus
# https://www.koordinaten-umrechner.de/decimal/48.237694,8.524661?karte=OpenStreetMap&zoom=16

daeurl = "models"

# Vestas V172, Bösingen
height_v_172 = 199
diameter_v_172 = 172

collada_wt1 = collada_wt.create_turbine(tower_height=height_v_172,
                    tower_bot_diameter = 6,
                    tower_top_diameter = 5,
                    nacelle_height = 3,
                    nacelle_length = 20,
                    nacelle_overhang = 8,
                    rotor_diameter = diameter_v_172,
                    blade_root_length = 3.5,
                    blade_root_diameter = 3,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )
collada_wt1.write(daeurl + '/vestas-v172.dae')

# Enercon E66/1500 (Nennleistung 1 500 kW, Durchmesser 66 m, Dunningen
height_e_66 = 95
diameter_e_66 = 66

collada_wt2 = collada_wt.create_turbine(tower_height=height_e_66,
                    tower_bot_diameter = 5,
                    tower_top_diameter = 4,
                    nacelle_height = 3,
                    nacelle_length = 10,
                    nacelle_overhang = 4,
                    rotor_diameter = diameter_e_66,
                    blade_root_length = 2.5,
                    blade_root_diameter = 2,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )
collada_wt2.write(daeurl + '/enercon-e-66.dae')

# Enercon E160, Herrenzimmern
height_e_160 = 166
diameter_e_160 = 160

collada_wt3 = collada_wt.create_turbine(tower_height=height_e_160,
                    tower_bot_diameter = 5,
                    tower_top_diameter = 4,
                    nacelle_height = 3,
                    nacelle_length = 15,
                    nacelle_overhang = 6,
                    rotor_diameter = diameter_e_160,
                    blade_root_length = 3,
                    blade_root_diameter = 2.5,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )
collada_wt3.write(daeurl + '/enercon-e-160.dae')

# Testturm RW
collada_zone = collada_wt.create_zone(
                    zone_height=246,
                    zone_diameter = 24.8,
                )
collada_zone.write(daeurl + '/testturm.dae')


In [4]:
# create kml for Google Earth
# https://www.schwarzwaelder-bote.de/inhalt.windenergieanlagen-in-boesingen-drei-plus-maximal-fuenf-im-gespraech.8840b7d3-7346-4f63-8c82-27f76a697bdd.html

# read coordinates from csv

import pandas as pd
import numpy as np

def csv2df(filename):
	df = pd.read_csv(filename, sep=',', decimal='.', header=0)
	df = df.dropna(axis=0, subset=['Latitude', 'Longitude'])
	df = df[df['Latitude'] != '']
	df = df[df['Longitude'] != '']

	df['Latitude'] = df['Latitude'].astype(np.float64)
	df['Longitude'] = df['Longitude'].astype(np.float64)

	return df

def add_turbine(row, folder,
	  	folder_model,
		folder_extent,
		folder_safety,
		folder_marker,
		markerstyle,
		color_extent=simplekml.Color.firebrick,
		visibility=1):

	name = row['Name']
	daemodel = row['Model']
	latitude = row['Latitude']
	longitude = row['Longitude']
	height = row['height']
	diameter = row['diameter']

	# add turbine or other model - will be shown in Google Earth Pro Desktop only
	turbine = folder_model.newmodel(name=name)
	turbine.link.href = 'models/' + daemodel
	turbine.location.latitude = latitude
	turbine.location.longitude = longitude
	turbine.scale.x = 1
	turbine.scale.y = 1
	turbine.scale.z = 1
	turbine.visibility = visibility
	turbine.orientation.heading = 90



	# add object as zylinder
	create_kml_cylinder (
		kmlfolder=folder_extent,
		cyl_center=(longitude, latitude),
		cyl_diameter=diameter,
		cyl_height=height + diameter/2,
		cyl_color=color_extent,
		cyl_name=name,
		cyl_visibility=visibility
	)

	# add safety zone as zylinder
	safety_distance = max(safety_distance_meter, diameter * safety_distance_factor)
	if folder_safety is not None:
		create_kml_cylinder (
			kmlfolder=folder_safety,
			cyl_center=(longitude, latitude),
			cyl_diameter=diameter + safety_distance*2,
			cyl_height=height + diameter/2 + safety_distance_top,
			cyl_color=simplekml.Color.lightgray,
			cyl_name=name,
			cyl_visibility=0
		)

	# add marker for turbine
	if folder_marker is not None and markerstyle is not None:
		location = folder_marker.newpoint(name=name, coords=[(longitude, latitude)])
		location.style=markerstyle
		location.visibility = 0

	return

# simple WEA locations with icon
markerstyle = simplekml.Style()
markerstyle.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle.iconstyle.color = simplekml.Color.darkgray

markerstyle_badenova = simplekml.Style()
markerstyle_badenova.labelstyle.color = simplekml.Color.lightgray  # Make the text blue
markerstyle_badenova.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_badenova.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_badenova.iconstyle.color = simplekml.Color.lightblue

markerstyle_iterra = simplekml.Style()
markerstyle_iterra.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle_iterra.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_iterra.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_iterra.iconstyle.color = simplekml.Color.red


# Badenova
folder_name = 'Badenova'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/badenova.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle_badenova, color_extent=simplekml.Color.firebrick, visibility=1), axis=1
)

# iTerra
folder_name = 'iTerra'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/iterra.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle_iterra, color_extent=simplekml.Color.darkorange, visibility=1), axis=1
)

# Dunningen
folder_name = 'Dunningen'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/dunningen.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle, color_extent=simplekml.Color.firebrick, visibility=1), axis=1
)

# Herrenzimmern
folder_name = 'Herrenzimmern'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte')
folder_marker.visibility = 0

df = csv2df('locations/herrenzimmern.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle, color_extent=simplekml.Color.firebrick, visibility=1), axis=1
)

# Testturm
folder_name = 'Testturm'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	None
folder_marker = None

df = csv2df('locations/testturm.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	None, color_extent=simplekml.Color.silver, visibility=1), axis=1
)


0    None
1    None
dtype: object

In [5]:
import math
import simplekml

# Erstellen Sie ein neues KML-Objekt
# kml = simplekml.Kml()
folder = kml.newfolder(name='Flugplatz')
airfield_boesingen = (8.534621455251553, 48.227697118500274)


#### Landebahn Bösingen

coordinates_text = """
8.531799515709411,48.22788047237495,0
8.531791237373254,48.22746570381057,0
8.537614083339131,48.22746248646123,0
8.537607267242425,48.22789309924998,0
8.531799515709411,48.22788047237495,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

pol = folder.newpolygon(
	name="Landebahn",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.blue)


#### Schutzzone Bösingen

# das Zentrum und den Radius des Kreises
center = airfield_boesingen
radius = 2000 / 111139  # Umwandlung von Metern in Grad

# Berechnen der Koordinaten des Kreises
coords = []
for i in range(360):
    theta = math.radians(i)
    x = center[0] + radius * math.cos(theta) / math.cos(math.radians(center[1]))  # Anpassung für Längenverzerrung
    y = center[1] + radius * math.sin(theta)
    coords.append((x, y, 0))  # Höhe 0, um den Kreis an die Oberfläche zu binden

#  Polygon, das den Kreis darstellt
pol = folder.newpolygon(
    name="Schutzzone",
    outerboundaryis=coords,
    altitudemode=simplekml.AltitudeMode.clamptoground
)
# Farbe des Polygons auf Rot und transparent setzen
pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)

# Linienfarbe auf Rot und die Linienbreite auf 5 setzen
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2


#### Endanflugbereich Bösingen

radius_m = 5000	# in Koordinaten-Einheiten (Grad)
height = 500.0

create_kml_cylinder (
	kmlfolder=folder,
    cyl_center=airfield_boesingen,
    cyl_diameter=radius_m * 2,
    cyl_height=height,
    cyl_color=simplekml.Color.green,
    cyl_name='Endanflugbereich',
	cyl_visibility=0
)



In [6]:
# Struktur mit Koordinaten: (Längengrad, Breitengrad, Höhe)
folder = kml.newfolder(name='Windvorranggebiete')

# Windvorranggebiet 1
coordinates_text = """
8.538625953140679,48.24733108376624,0
8.545398800655983,48.25155668096118,0
8.545061726012847,48.255504959821,0
8.538271166860046,48.25418519218096,0
8.521300387225741,48.25642433277036,0
8.519359663435308,48.24964241078206,0
8.523113634981584,48.25018720725038,0
8.524434812536384,48.2480791750467,0
8.521744597138214,48.24594686097112,0
8.522702190206415,48.24477797568368,0
8.525950765261815,48.2442144809804,0
8.53804803344558,48.24392246020017,0
8.538625953140679,48.24733108376624,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 1",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 1"


# Windvorranggebiet 2
coordinates_text = """
8.514070913418958,48.26142175265778,0
8.514622479179902,48.26552246326965,0
8.527729663701306,48.26672900175677,0
8.531674441607455,48.26384212287596,0
8.532523110802014,48.26421272078534,0
8.530014534666856,48.26701405884985,0
8.525004254661457,48.27837712280289,0
8.520348560492131,48.27584428565216,0
8.512373126115014,48.27177096214482,0
8.506707838964472,48.26880690993875,0
8.507707710669782,48.2564529695051,0
8.517445889736155,48.25661666038091,0
8.514070913418958,48.26142175265778,0
"""

# Teilen Sie den Text in eine Liste von Zeilen
lines = coordinates_text.strip().split('\n')

# Wandeln Sie jede Zeile in ein Tupel von Zahlen um
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 2",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 2"

# Windvorranggebiet 3
coordinates_text = """
8.5522629907101,48.26321448744435,0
8.552258376763195,48.25989643569763,0
8.553128098300105,48.25368698104617,0
8.558522604953403,48.25027521335589,0
8.561505433168408,48.25019772319624,0
8.564563501824072,48.25381918640282,0
8.564232916284588,48.25828634025782,0
8.559161166027341,48.26434588502315,0
8.559146231551225,48.26700202858952,0
8.554157230908018,48.26591884900093,0
8.554485660548242,48.27367261632118,0
8.548681751883251,48.27471567553113,0
8.547827630360242,48.27003625407925,0
8.5522629907101,48.26321448744435,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 3",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 3"


# Windvorranggebiet 4
coordinates_text = """
8.566613951316528,48.21751577350571,0
8.563450873050797,48.21486175121698,0
8.552957993518984,48.20980641549801,0
8.556186048885095,48.20665602637702,0
8.561204273953242,48.20732363941927,0
8.565200989995743,48.20424968308848,0
8.569392397898296,48.20868686765963,0
8.574446120720101,48.21026664619694,0
8.566613951316528,48.21751577350571,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 4",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 4"



In [7]:
def create_kml_slope(kml_folder, start_lat, start_lon, start_altitude, direction, length, slope, width, color=simplekml.Color.yellow, name="slope"):
    import math
    import simplekml

    transparency = 120

    # Umrechnung von Grad in Radiant
    direction_rad = math.radians(direction)
    slope_rad = math.radians(slope)

    # Berechnung der Höhe und der horizontalen Entfernung
    height = length * math.sin(slope_rad)  # in Kilometern
    distance = length * math.cos(slope_rad)  # in Kilometern

    # Verschiebung um die halbe Breite
    half_width = width / 2
    start_lat -= half_width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    start_lon -= half_width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

    # Berechnung der Endkoordinaten
    end_lat = start_lat + distance * math.cos(direction_rad) / 111.32  # Umrechnung von Kilometern in Grad
    end_lon = start_lon + distance * math.sin(direction_rad) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad
    end_altitude = start_altitude + height * 1000  # Umrechnung von Kilometern in Metern

    # Berechnung der Breitenkoordinaten
    width_lat1 = start_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    width_lon1 = start_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

    width_lat2 = end_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    width_lon2 = end_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(end_lat)))  # Umrechnung von Kilometern in Grad

    # Erstellen Sie die Eckpunkte der Fläche
    coordinates = [
        (start_lon, start_lat, start_altitude),
        (end_lon, end_lat, end_altitude),
        (width_lon2, width_lat2, end_altitude),
        (width_lon1, width_lat1, start_altitude),
    ]

    # Fügen Sie das Polygon zum KML-Ordner hinzu
    pol = kml_folder.newpolygon(name=name, outerboundaryis=coordinates, altitudemode=simplekml.AltitudeMode.absolute)
    pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
    pol.style.linestyle.color = simplekml.Color.changealphaint(transparency, color)


In [8]:
def create_kml_cube(kml_folder, start_lat, start_lon, start_altitude, direction, length, slope, width, color=simplekml.Color.yellow, name="cube"):

	import math
	import simplekml

	cube_folder = kml_folder.newfolder(name='Abflugbereich')

	cubehight = 100
	transparency = 80

	# Umrechnung von Grad in Radiant
	direction_rad = math.radians(direction)
	slope_rad = math.radians(slope)

	# Berechnung der Höhe und der horizontalen Entfernung
	height = length * math.sin(slope_rad)  # in Kilometern
	distance = length * math.cos(slope_rad)  # in Kilometern

	# Verschiebung um die halbe Breite
	half_width = width / 2
	start_lat -= half_width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	start_lon -= half_width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

	# Berechnung der Endkoordinaten
	end_lat = start_lat + distance * math.cos(direction_rad) / 111.32  # Umrechnung von Kilometern in Grad
	end_lon = start_lon + distance * math.sin(direction_rad) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad
	end_altitude = start_altitude + height * 1000  # Umrechnung von Kilometern in Metern

	# Berechnung der Breitenkoordinaten
	width_lat1 = start_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	width_lon1 = start_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

	width_lat2 = end_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	width_lon2 = end_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(end_lat)))  # Umrechnung von Kilometern in Grad

	# Erstellen Sie die Eckpunkte der unteren und oberen Fläche
	lower_coordinates = [
		(start_lon, start_lat, start_altitude),
		(end_lon, end_lat, end_altitude),
		(width_lon2, width_lat2, end_altitude),
		(width_lon1, width_lat1, start_altitude),
	]

	upper_coordinates = [
		(start_lon, start_lat, start_altitude + cubehight),
		(end_lon, end_lat, end_altitude + cubehight),
		(width_lon2, width_lat2, end_altitude + cubehight),
		(width_lon1, width_lat1, start_altitude + cubehight),
	]

	# Erstellen Sie die Eckpunkte der seitlichen Flächen
	side1_coordinates = [
		lower_coordinates[0],
		upper_coordinates[0],
		upper_coordinates[1],
		lower_coordinates[1],
	]

	side2_coordinates = [
		lower_coordinates[1],
		upper_coordinates[1],
		upper_coordinates[2],
		lower_coordinates[2],
	]

	side3_coordinates = [
		lower_coordinates[2],
		upper_coordinates[2],
		upper_coordinates[3],
		lower_coordinates[3],
	]

	side4_coordinates = [
		lower_coordinates[3],
		upper_coordinates[3],
		upper_coordinates[0],
		lower_coordinates[0],
	]

	# Fügen Sie die Polygone zum KML-Ordner hinzu
	lower_pol = cube_folder.newpolygon(name=name, outerboundaryis=lower_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	lower_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	lower_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	upper_pol = cube_folder.newpolygon(name=name, outerboundaryis=upper_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	upper_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	upper_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side1_pol = cube_folder.newpolygon(name=name, outerboundaryis=side1_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side1_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side1_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side2_pol = cube_folder.newpolygon(name=name, outerboundaryis=side2_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side2_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side2_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side3_pol = cube_folder.newpolygon(name=name, outerboundaryis=side3_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side3_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side3_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side4_pol = cube_folder.newpolygon(name=name, outerboundaryis=side4_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side4_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side4_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	cube_folder.visibility = 0

	return


In [9]:

def calculate_slope(speed_kmh, climb_rate_ms):
    # Umwandlung der Geschwindigkeit von km/h in m/s
    speed_ms = speed_kmh * 1000 / 3600

    # Berechnung der Steigung
    slope_rad = math.atan(climb_rate_ms / speed_ms)

    # Umwandlung der Steigung von Radiant in Grad
    slope_deg = math.degrees(slope_rad)

    return slope_deg


In [10]:
#### Abflug Bösingen

folder = kml.newfolder(name='Flugrouten')

# Startkoordinaten
lon, lat = airfield_boesingen


# Platzrunde auf Basis von Koordinaten aus areas/platzrunde.csv
df = csv2df('areas/platzrunde.csv')

# Polygon für Platzrunde aus df
coordinates = df[['Latitude', 'Longitude', 'Altitude']].values.tolist()
pol = folder.newpolygon(name='Platzrunde', outerboundaryis=coordinates, altitudemode=simplekml.AltitudeMode.absolute)
pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.green)
pol.style.linestyle.color = simplekml.Color.changealphaint(100, simplekml.Color.red)
pol.linestyle.width = 4
#pol.linestyle.outercolor = simplekml.Color.changealphaint(100, simplekml.Color.green)
pol.visibility = 1




# typischer Anflug Bösingen
start_altitude = 700  # in Metern
slope = calculate_slope(60,1)	# 60 km/h bei 1 m/s
length = 3  # Länge des Quaders in Kilometern
width = 0.3  # Breite des Quaders in Kilometern
direction = 270
create_kml_slope(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.yellow, "Anflug 27")

direction = 90
create_kml_slope(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.yellow, "Anflug 09")

# typischer Abflug im Schleppverbund
start_altitude = 700  # in Metern
slope = calculate_slope(60,1.5)	# 60 km/h bei 1.5 m/s
length = 5  # Länge des Quaders in Kilometern
width = 2  # Breite des Quaders in Kilometern
direction = 0
create_kml_cube(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.blue, "Abflug")


In [11]:
# read provided kml xc files and extract launch phase
folder = kml.newfolder(name='Tracks.KML')

def create_kml_line(kml_folder, coordinates, flight_name, flight_url, color=simplekml.Color.lightblue, transparency=100):
	ls = kml_folder.newlinestring(name=flight_name)
	ls.coords = coordinates
	ls.extrude = 0
	ls.altitudemode = simplekml.AltitudeMode.absolute
	ls.style.linestyle.width = 2
	ls.style.linestyle.color = color
	ls.description = f'<a href="{flight_url}">Link</a>'  # Add URL to the description
	ls.visibility = 1

	return

# process all kml files in tracks folder
import glob

kml_folder = 'tracks/kml'
kml_files = glob.glob(kml_folder + '/*.kml')

for kml_path in kml_files:
	# extract launch phase
	# print(kml_path)
	track, flight_url, flight_name = kml_extract_launch_phase(kml_path, min_altitude=710, max_altitude=1500, min_climb_rate=0)

	if track is not None:	# only add if track is not empty
		coordinates = track[['Longitude', 'Latitude', 'Altitude']].values.tolist()

	# check if coordinates are within the area
	if (coordinates[0][1] < 48.2) or (coordinates[0][1] > 48.2):
		print(flight_name, coordinates[0][1], coordinates[0][0])
	elif (coordinates[0][0] < 8.5) or (coordinates[0][0] > 8.5):
		print(flight_name, coordinates[0][1], coordinates[0][0])
	else:
		create_kml_line(folder, coordinates, flight_name, flight_url, simplekml.Color.purple)


tracks/kml/2021_1434386.kml: Release towline at index 95 with altitude 1480.0
1434386 48.2276 8.5361166666667
tracks/kml/2022_1576061.kml: Release towline at index 63 with altitude 1472.0
1576061 48.2277 8.5351166666667
tracks/kml/2023_1741037.kml: Release towline at index 103 with altitude 1457.0
1741037 48.227733333333 8.5354833333333
tracks/kml/2022_1546648.kml: Release towline at index 53 with altitude 1363.0
1546648 48.227766666667 8.5336166666667
tracks/kml/2019_1165304.kml: Release towline at index 94 with altitude 1321.0
1165304 48.228366666667 8.5403
tracks/kml/2019_1122532.kml: Release towline at index 68 with altitude 1434.0
1122532 48.227983333333 8.5394166666667
tracks/kml/2021_1362547.kml: Release towline at index 60 with altitude 1437.0
1362547 48.2275 8.5355666666667
tracks/kml/2022_1546688.kml: Release towline at index 83 with altitude 1434.0
1546688 48.2276 8.5332166666667
tracks/kml/2019_1164603.kml: Release towline at index 80 with altitude 1437.0
1164603 48.2277 8.

In [12]:
# read provided kml xc files and extract launch phase
folder = kml.newfolder(name='Tracks.IGC')

def create_kml_line(igc_folder, coordinates, flight_name, flight_url, color=simplekml.Color.lightblue, transparency=100):
	ls = igc_folder.newlinestring(name=flight_name)
	ls.coords = coordinates
	ls.extrude = 0
	ls.altitudemode = simplekml.AltitudeMode.absolute
	ls.style.linestyle.width = 2
	ls.style.linestyle.color = color
	ls.description = f'{flight_name}'
	ls.visibility = 1

	return

# process all kml files in tracks folder
import glob

igc_folder = 'tracks/igc'
igc_files = glob.glob(igc_folder + '/*.igc')

for igc_path in igc_files:
	# extract launch phase
	# print(igc_path)
	track, flight_url, flight_name = igc_extract_launch_phase(igc_path, min_altitude=710, max_altitude=1500, min_climb_rate=0)

	if track is not None:	# only add if track is not empty
		coordinates = track[['Longitude', 'Latitude', 'Altitude']].values.tolist()
		create_kml_line(folder, coordinates, flight_name, flight_url, simplekml.Color.mediumpurple)


Single IGC file supplied: tracks/igc/2020-07-25_15.46_86_BA_pp_singen_Flugplatz.igc
Header (not implemented): FPLTPILOT:Rainer Thieringer
Header (not implemented): FGTYGLIDERTYPE:AIR Atos VQ Race
Header (not implemented): FDTM100GPSDATUM:WGS-84
Header (not implemented): FCIDCOMPETITIONID:0000
Header (not implemented): FCCLCOMPETITION CLASS:HG Rigid
Header (not implemented): FSITSITE:86 Bösingen Flugplatz
Flight date: 2020-07-25
Record Type L not implemented: XSX;GF:1.5
Record Type G not implemented: D520AC8DBE3A18810BC571301FE7DF06
tracks/igc/2020-07-25_15.46_86_BA_pp_singen_Flugplatz.igc: Release towline at index 81 with altitude 943
Single IGC file supplied: tracks/igc/2018-08-19_14.17_Sonderlandeplatz_Boesingen.igc
Header (not implemented): OPLTPILOT:Rainer Thieringer
Header (not implemented): OGTYGLIDERTYPE:AIR Atos VQ Race
Header (not implemented): OCIDCOMPETITIONID:0000
Header (not implemented): ODTM100GPSDATUM:WGS-84
Header (not implemented): OCCLCOMPETITION CLASS:HG Rigid
Heade

In [13]:
# Read CSV Files with coordinates and create KML

folder = kml.newfolder(name='Tracks.CSV')


# read coordinates from csv
df = csv2df('tracks/start0001.csv')
coordinates = df[['Longitude', 'Latitude', 'Altitude']].values.tolist()

print (df.head())


        Datetime (UTC)  Elapsed Time   Latitude  Longitude  Altitude  \
0  2023-09-24 11:28:26           0.0  48.227717   8.532133       692   
1  2023-09-24 11:28:27           1.0  48.227717   8.532133       692   
2  2023-09-24 11:28:28           2.0  48.227717   8.532133       692   
3  2023-09-24 11:28:29           3.0  48.227717   8.532133       692   
4  2023-09-24 11:28:30           4.0  48.227717   8.532133       692   

   Distance Delta  Distance Total  Groundspeed  Groundspeed Peak  \
0             0.0             0.0          0.0               0.0   
1             0.0             0.0          0.0               0.0   
2             0.0             0.0          0.0               0.0   
3             0.0             0.0          0.0               0.0   
4             0.0             0.0          0.0               0.0   

   Altitude Delta (GPS)  Altitude Delta (Pressure)  Climb Speed  Climb Total  \
0                     0                          0          0.0            0  

In [14]:
# save models to kml

kml_file = './output/WEA-boesingen.kml'
kml.save(kml_file)

In [15]:
# create kmz from kml and dae files

import zipfile

def create_kmz(kml_filename, dae_filenames, output_filename):
    with zipfile.ZipFile(output_filename, 'w') as kmz:
        kmz.write(kml_filename)

        for dae in dae_filenames:
            kmz.write(dae, os.path.join("models", os.path.basename(dae)))

# Listet alle .dae-Dateien im "model"-Verzeichnis auf

dae_files = [os.path.join("models", f) for f in os.listdir("models") if f.endswith('.dae')]
kml_file = "./output/WEA-boesingen.kml"  # Ersetzen Sie dies durch den Pfad zu Ihrer KML-Datei
output_file = "./output/WEA-boesingen.kmz"
create_kmz(kml_file, dae_files, output_file)


## Quellen

* https://www.landesrecht-bw.de/jportal/?quelle=jlink&docid=JURE060017391&psml=bsbawueprod.psml&max=true&doc.part=L&doc.norm=all
* https://www.daec.de/media/files/2022/Fachbereiche/Umwelt/Leitfaden_Luftfahrthindernisse_14_03_2022.pdf
* https://www.fsco.de/images/Dokumente/Studie_DLR.pdf
* https://www.dhv.de/fileadmin/user_upload/aktuell_zu_halten/Gelaende/DHV_info_199_windkraft.pdf
* http://www.pontepress.de/pdf/u12_201706.pdf
* [Beteiligungsverfahren Teilplan "Regionalbedeutsame Windkraftanlagen"](https://www.regionalverband-sbh.de/seite/653796/beteiligungsverfahren-teilplan-regionalbedeutsame-windkraftanlagen.html)
* [Protokoll / Präsentationen Windpark Bösingen](https://www.boesingen.de/de/Aktuelles/Gemeindenachrichten/Gemeindenachricht?view=publish&item=article&id=1309)
* [Protokoll / Präsentationen Windpark Herrenzimmern](https://www.boesingen.de/de/Aktuelles/Gemeindenachrichten/Gemeindenachricht?view=publish&item=article&id=1308)



## Regelungen

* [Abstandsempfehlungen zu Wohn- und Mischgebieten](https://fachagentur-windenergie.de/fileadmin/files/Veroeffentlichungen/Planung/FA_Wind_Abstandsempfehlungen_Aktualisierung_3-2023.pdf): 700m in BaWü
* [Abstand WKA DHV](https://www.dhv.de/piloteninfos/gelaende-luftraum-natur/fluggelaendeflugbetrieb/flugbetrieb/windkraftanlagen-und-fluggelaende/mindestabstand-laut-nfl/): 600m 
* [NfL 92/13](https://www.dhv.de/fileadmin/user_upload/files/2015/05/NfL_-_Abstand_zu_Anlagen.pdf): 400 Gegenanflug, sonst 850
* [Raumnutzungskarte](./papers/Anlage_2.1_Raumnutzungskarte_Umsetzung_Landesflaechenziel_nord_Beilage_25_2023_TOP_5.pdf)

## Untersuchungen

* [Wissenschaftlicher Dienst des Deutschen Bundestag WD 8 - 3000 - 007/22
Zu Wirbelschleppen von Windparks](https://www.bundestag.de/resource/blob/919868/18b8e759ff8a4b782befd451138d4003/WD-8-007-22-pdf-data.pdf)
* [Schreckgespenst Turbulenzen](./papers/Schreckgespenst_Turbulenz_v2.pdf)

## Presse 

* https://www.schwarzwaelder-bote.de/inhalt.dunningen-windkraft-projekt-liegt-erst-mal-auf-eis.ed5bc4b7-42df-470d-a7cb-97cd0315cbbd.html


## Flurstücke in Herrenzimmern, Fa. Alterric

* https://www.schwarzwaelder-bote.de/inhalt.windpark-in-herrenzimmern-anlage-wird-mit-rotorblatt-246-meter-hoch.ef522d07-0ce3-4fe3-a14e-e5a5bc19a655.html
* ENERCON E-160 https://www.enercon.de/de/windanlagen/e-160-ep5
* Gewann Saugrube
* Sponäcker
* Waldflurstück in Kleinheide


## Einschätzungen 

